In [ ]:
import torch
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import zipfile


import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import cv2
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.utils import to_categorical
from keras import backend as k
%matplotlib inline
import pickle
from keras.utils import to_categorical
from keras.layers import Dense, Input, Conv2D, Flatten, MaxPooling2D, Activation, Dropout, BatchNormalization
from keras.layers import LeakyReLU
from keras.optimizers import Adamax
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras import backend as K





In [ ]:
RESIZE_DIM=32
FIG_WIDTH=20
HEIGHT_PER_ROW=3


In [ ]:
import zipfile

zip_file_path = '/content/drive/MyDrive/dataset/handw_bangla.zip'
output_dir = '/content/unzip'

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Extract the contents of the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print("ZIP file extracted successfully.")


ZIP file extracted successfully.


In [ ]:

data_dir=os.path.join('/content/unzip')
paths_train_a=glob.glob(os.path.join(data_dir,'training-a','*.png'))
paths_train_b=glob.glob(os.path.join(data_dir,'training-b','*.png'))
paths_train_e=glob.glob(os.path.join(data_dir,'training-e','*.png'))
paths_train_c=glob.glob(os.path.join(data_dir,'training-c','*.png'))
paths_train_d=glob.glob(os.path.join(data_dir,'training-d','*.png'))
paths_train_all=paths_train_a+paths_train_b+paths_train_c+paths_train_d+paths_train_e



paths_test_a=glob.glob(os.path.join(data_dir,'testing-a','*.png'))
paths_test_b=glob.glob(os.path.join(data_dir,'testing-b','*.png'))
paths_test_e=glob.glob(os.path.join(data_dir,'testing-e','*.png'))
paths_test_c=glob.glob(os.path.join(data_dir,'testing-c','*.png'))
paths_test_d=glob.glob(os.path.join(data_dir,'testing-d','*.png'))
paths_test_f=glob.glob(os.path.join(data_dir,'testing-f','*.png'))+glob.glob(os.path.join(data_dir,'testing-f','*.JPG'))
paths_test_auga=glob.glob(os.path.join(data_dir,'testing-auga','*.png'))
paths_test_augc=glob.glob(os.path.join(data_dir,'testing-augc','*.png'))
paths_test_all=paths_test_a+paths_test_b+paths_test_c+paths_test_d+paths_test_e+paths_test_f+paths_test_auga+paths_test_augc

path_label_train_a=os.path.join(data_dir,'training-a.csv')
path_label_train_b=os.path.join(data_dir,'training-b.csv')
path_label_train_e=os.path.join(data_dir,'training-e.csv')
path_label_train_c=os.path.join(data_dir,'training-c.csv')
path_label_train_d=os.path.join(data_dir,'training-d.csv')





In [ ]:
def get_key(path):
    # Separates the key of an image from the filepath
    key = path.split(sep=os.sep)[-1]
    return key

def get_data(paths_img, path_label=None, resize_dim=None):
    '''Reads images from the filepaths, resizes them (if given), and returns them in a numpy array.
    Args:
        paths_img: image filepaths
        path_label: pass image label filepaths while processing training data, defaults to None while processing testing data
        resize_dim: if given, the image is resized to resize_dim x resize_dim (optional)
    Returns:
        X: group of images
        y: categorical true labels
    '''
    X = []  # Initialize empty list for resized images
    for i, path in enumerate(paths_img):
        img = cv2.imread(path, cv2.IMREAD_COLOR)  # Load images in color (BGR)

        if resize_dim is not None:
            img = cv2.resize(img, (resize_dim, resize_dim), interpolation=cv2.INTER_AREA)  # Resize image to resize_dim x resize_dim
        X.append(img)  # Append image to the list

        # Display progress
        if i == len(paths_img) - 1:
            end = '\n'
        else:
            end = '\r'
        print('Processed {}/{}'.format(i + 1, len(paths_img)), end=end)

    X = np.array(X)  # Transform list to numpy array
    if path_label is None:
        return X
    else:
        df = pd.read_csv(path_label)  # Read labels
        df = df.set_index('filename')
        y_label = [df.loc[get_key(path)]['digit'] for path in paths_img]  # Get the labels corresponding to the images
        y = to_categorical(y_label, 10)  # Transform integer value to categorical variable
        return X, y
def imshow_group(X, y, y_pred=None, n_per_row=10, phase='processed'):
    n_sample = len(X)
    img_dim = X.shape[1]
    j = np.ceil(n_sample / n_per_row)
    fig = plt.figure(figsize=(FIG_WIDTH, HEIGHT_PER_ROW * j))
    for i, img in enumerate(X):
        plt.subplot(j, n_per_row, i + 1)
        plt.imshow(img)
        if phase == 'processed':
            plt.title(np.argmax(y[i]))
        if phase == 'prediction':
            top_n = 3  # top 3 predictions with highest probabilities
            ind_sorted = np.argsort(y_pred[i])[::-1]
            h = img_dim + 4
            for k in range(top_n):
                string = 'pred: {} ({:.0f}%)\n'.format(ind_sorted[k], y_pred[i, ind_sorted[k]] * 100)
                plt.text(img_dim / 2, h, string, horizontalalignment='center', verticalalignment='center')
                h += 4
            if y is not None:
                plt.text(img_dim / 2, -4, 'true label: {}'.format(np.argmax(y[i])),
                         horizontalalignment='center', verticalalignment='center')
        plt.axis('off')
    plt.show()

def create_submission(predictions, keys, path):
    result = pd.DataFrame(
        predictions,
        columns=['label'],
        index=keys
    )
    result.index.name = 'key'
    result.to_csv(path, index=True)
# Rest of the code...


In [ ]:
X_train_a,y_train_a=get_data(paths_train_a,path_label_train_a,resize_dim=RESIZE_DIM)
X_train_b,y_train_b=get_data(paths_train_b,path_label_train_b,resize_dim=RESIZE_DIM)
X_train_c,y_train_c=get_data(paths_train_c,path_label_train_c,resize_dim=RESIZE_DIM)
X_train_d,y_train_d=get_data(paths_train_d,path_label_train_d,resize_dim=RESIZE_DIM)
X_train_e,y_train_e=get_data(paths_train_e,path_label_train_e,resize_dim=RESIZE_DIM)


Processed 19702/19702
Processed 359/359
Processed 24298/24298
Processed 10908/10908
Processed 16778/16778


In [ ]:
X_train_all=np.concatenate((X_train_a,X_train_b,X_train_c,X_train_d,X_train_e),axis=0)
y_train_all=np.concatenate((y_train_a,y_train_b,y_train_c,y_train_d,y_train_e),axis=0)
X_train_all.shape, y_train_all.shape





((72045, 32, 32, 3), (72045, 10))

In [ ]:
# plt.subplot(221)
# plt.imshow(X_train_all[0], cmap=plt.get_cmap('gray'))
# plt.subplot(222)
# plt.imshow(X_train_all[1], cmap=plt.get_cmap('gray'))
# plt.subplot(223)
# plt.imshow(X_train_all[2], cmap=plt.get_cmap('gray'))
# plt.subplot(224)
# plt.imshow(X_train_all[3], cmap=plt.get_cmap('gray'))
# # show the plot
# plt.show()

In [ ]:
# imgg=X_train_all[1]
# hist = cv2.calcHist([imgg],[0],None,[256],[0,256])
# plt.hist(imgg.ravel(),256,[0,256])

# # show the plotting graph of an image

# plt.show()

In [ ]:
X_test_a=get_data(paths_test_a,resize_dim=RESIZE_DIM)
X_test_b=get_data(paths_test_b,resize_dim=RESIZE_DIM)
X_test_c=get_data(paths_test_c,resize_dim=RESIZE_DIM)
X_test_d=get_data(paths_test_d,resize_dim=RESIZE_DIM)
X_test_e=get_data(paths_test_e,resize_dim=RESIZE_DIM)
X_test_f=get_data(paths_test_f,resize_dim=RESIZE_DIM)
X_test_auga=get_data(paths_test_auga,resize_dim=RESIZE_DIM)
X_test_augc=get_data(paths_test_augc,resize_dim=RESIZE_DIM)


Processed 3489/3489
Processed 69/69
Processed 4381/4381
Processed 1948/1948
Processed 2970/2970
Processed 495/495
Processed 2168/2168
Processed 2106/2106


In [ ]:
X_test_all=np.concatenate((X_test_a,X_test_b,X_test_c,X_test_d,X_test_e,X_test_f,X_test_auga,X_test_augc))

In [ ]:
X_tshow_all=X_test_all
X_tshow_all.shape

(17626, 32, 32, 3)

In [ ]:
 #X_train_all = X_train_all/255
#X_test_all=X_test_all/255

In [ ]:
##X_train_all = X_train_all.reshape(X_train_all.shape[0],32, 32,1).astype('float32')
#X_test_all = X_test_all.reshape(X_test_all.shape[0],32, 32,1).astype('float32')

In [ ]:
X_train_all.shape

(72045, 32, 32, 3)

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torchvision.transforms as transforms
# import torchvision.models as models
# from torch.utils.data import Dataset, DataLoader

In [ ]:
# from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# label_encoder = LabelEncoder()
# y_train_all_encoded = label_encoder.fit_transform(np.argmax(y_train_all, axis=1))
# y_train_all_reshaped = y_train_all_encoded.reshape(-1, 1)
# onehot_encoder = OneHotEncoder(sparse=False)
# y_train_all_onehot = onehot_encoder.fit_transform(y_train_all_reshaped)

# print(y_train_all_encoded[:10])

# y_train_all_onehot.shape

In [ ]:
# # Define the transformation
# transform = transforms.Compose([
#     transforms.ToPILImage(),                    # Convert the array to PIL Image
#     transforms.Resize((224, 224)),               # Resize the images to (224, 224)
#     transforms.ToTensor(),                       # Convert the images to tensors
#     transforms.Normalize(                        # Normalize the image tensors
#         mean=[0.485, 0.456, 0.406],               # Mean values for normalization
#         std=[0.229, 0.224, 0.225]                 # Standard deviation values for normalization
#     )
# ])

In [ ]:
# # Define your custom dataset class
# class CustomDataset(Dataset):
#     def __init__(self, data, targets, transform=None):
#         self.data = data
#         self.targets = targets
#         self.transform = transform

#     # def __len__(self):
#         return len(self.data)

#     def __getitem__(self, index):
#         x = self.data[index]
#         y = self.targets[index]

#         if self.transform:
#             x = self.transform(x)

#         return x, y


# # Convert your NumPy arrays to PyTorch datasets
# train_dataset = CustomDataset(X_train_all, y_train_all, transform=transform)
# test_dataset = CustomDataset(X_test_all, None, transform=transform)  # No labels for testing dataset

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader



In [ ]:
# Define the transformation
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Define your custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        if self.transform:
            x = self.transform(x)

        return x, y




In [ ]:
# Convert your NumPy arrays to PyTorch datasets
train_dataset = CustomDataset(X_train_all, y_train_all, transform=transform)
test_dataset = CustomDataset(X_test_all, None, transform=transform)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Use ResNet-15
model_15 = models.resnet18(pretrained=True)
num_classes = 10  # Number of output classes
model_15.fc = nn.Linear(model_15.fc.in_features, num_classes)  # Replace the fully connected layer

# Use ResNet-50
model_50 = models.resnet50(pretrained=True)
model_50.fc = nn.Linear(model_50.fc.in_features, num_classes)  # Replace the fully connected layer

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer_15 = optim.Adam(model_15.parameters(), lr=0.001)
optimizer_50 = optim.Adam(model_50.parameters(), lr=0.001)



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 78.4MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed 

In [ ]:
# Training loop for ResNet-15
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_15.to(device)
for epoch in range(num_epochs):
    model_15.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer_15.zero_grad()

        outputs = model_15(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_15.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_dataset)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}')



Epoch 1/10, Loss: 0.1147
Epoch 2/10, Loss: 0.0578
Epoch 3/10, Loss: 0.0458
Epoch 4/10, Loss: 0.0381
Epoch 5/10, Loss: 0.0310
Epoch 6/10, Loss: 0.0268
Epoch 7/10, Loss: 0.0218
Epoch 8/10, Loss: 0.0190
Epoch 9/10, Loss: 0.0152
Epoch 10/10, Loss: 0.0135


In [ ]:
# Save ResNet-15 model
save_path_resnet15 = '/content/15.pth'
torch.save(model_15.state_dict(), save_path_resnet15)


In [ ]:
# Training loop for ResNet-50
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_50.to(device)
for epoch in range(num_epochs):
    model_50.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer_50.zero_grad()

        outputs = model_50(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_50.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_dataset)
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}')
    # Save ResNet-50 model





NameError: ignored

In [ ]:

save_path_resnet50 = '/content/16.pth'
torch.save(model_50.state_dict(), save_path_resnet50)


In [ ]:
# Evaluation
model_15.eval()
model_50.eval()
with torch.no_grad():
    correct_15 = 0
    correct_50 = 0
    total = 0

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs_15 = model_15(images)
        _, predicted_15 = torch.max(outputs_15, dim=1)
        correct_15 += (predicted_15 == labels).sum().item()

        outputs_50 = model_50(images)
        _, predicted_50 = torch.max(outputs_50, dim=1)
        correct_50 += (predicted_50 == labels).sum().item()

        total += labels.size(0)

    accuracy_15 = correct_15 / total
    accuracy_50 = correct_50 / total

    print(f'Accuracy of ResNet-15: {accuracy_15:.4f}')
    print(f'Accuracy of ResNet-50: {accuracy_50:.4f}')

NameError: ignored

In [ ]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset

# Define your custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        if self.transform:
            x = self.transform(x)

        return x, y

# Convert your NumPy arrays to PyTorch datasets
train_dataset = CustomDataset(X_train_all, y_train_all, transform=transform)


In [ ]:
import torch
import torchvision.transforms as transforms

# Define the transformation
transform = transforms.Compose([
    transforms.ToPILImage(),                      # Convert NumPy array to PIL Image
    transforms.Resize((32, 32)),                   # Resize the images to the desired size
    transforms.ToTensor()                          # Convert the images to tensors
])

# Apply the transformation to your dataset
X_train_all_transformed = torch.stack([transform(img) for img in X_train_all])
X_test_all_transformed = torch.stack([transform(img) for img in X_test_all])

# Normalize the image tensors using NumPy operations
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
X_train_all_normalized = (X_train_all_transformed - mean[:, None, None]) / std[:, None, None]
X_test_all_normalized = (X_test_all_transformed - mean[:, None, None]) / std[:, None, None]


In [ ]:
#alexnet = models.alexnet(pretrained=True)


In [ ]:
resnet15 = models.resnet152(pretrained=True)


In [ ]:
resnet50 = models.resnet50(pretrained=True)


In [ ]:
num_classes = 10  # Replace with the actual number of classes in your dataset

#alexnet.classifier[6] = nn.Linear(4096, num_classes)
resnet15.fc = nn.Linear(resnet15.fc.in_features, num_classes)
resnet50.fc = nn.Linear(resnet50.fc.in_features, num_classes)


In [ ]:
criterion = nn.CrossEntropyLoss()
#optimizer_alexnet = optim.SGD(alexnet.parameters(), lr=0.001, momentum=0.9)
optimizer_resnet15 = optim.SGD(resnet15.parameters(), lr=0.001, momentum=0.9)
optimizer_resnet50 = optim.SGD(resnet50.parameters(), lr=0.001, momentum=0.9)


In [ ]:
X_train_all_tensor = torch.from_numpy(X_train_all)
y_train_all_tensor = torch.from_numpy(y_train_all)
X_test_all_tensor = torch.from_numpy(X_test_all)


In [ ]:
train_dataset = torch.utils.data.TensorDataset(X_train_all_tensor, y_train_all_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# Assuming you have test labels, create a test dataset and data loader as well
test_dataset = torch.utils.data.TensorDataset(X_test_all_tensor)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:
X_train_all.shape

(72045, 32, 32, 3)

In [ ]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((32, 32)),
    transforms.Grayscale(num_output_channels=3),  # Convert to grayscale with 3 channels
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#alexnet.to(device)
resnet15.to(device)
resnet50.to(device)

num_epochs = 10  # Adjust the number of epochs as needed

for epoch in range(num_epochs):
    # Training loop
    #alexnet.train()
    resnet15.train()
    resnet50.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        # Zero the gradients
        #optimizer_alexnet.zero_grad()
        optimizer_resnet15.zero_grad()
        optimizer_resnet50.zero_grad()

        # Forward pass
        #output_alexnet = alexnet(data)
        output_resnet15 = resnet15(data)
        output_resnet50 = resnet50(data)

        # Calculate loss
        #loss_alexnet = criterion(output_alexnet, target)
        loss_resnet15 = criterion(output_resnet15, target)
        loss_resnet50 = criterion(output_resnet50, target)

        # Backward pass
        #loss_alexnet.backward()
        loss_resnet15.backward()
        loss_resnet50.backward()

        # Update weights
        #optimizer_alexnet.step()
        optimizer_resnet15.step()
        optimizer_resnet50.step()

        # Print training progress
        if (batch_idx+1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

    # Evaluation loop (optional, if you have test labels)
    #alexnet.eval()
    resnet15.eval()
    resnet50.eval()

    with torch.no_grad():
        #correct_alexnet = 0
        correct_resnet15 = 0
        correct_resnet50 = 0
        total = 0

        for data in test_loader:
            data = data.to(device)

            #output_alexnet = alexnet(data)
            output_resnet15 = resnet15(data)
            output_resnet50 = resnet50(data)

          #  _, predicted_alexnet = torch.max(output_alexnet.data, 1)
            _, predicted_resnet15 = torch.max(output_resnet15.data, 1)
            _, predicted_resnet50 = torch.max(output_resnet50.data, 1)

            total += data.size(0)
            #correct_alexnet += (predicted_alexnet == target).sum().item()
            correct_resnet15 += (predicted_resnet15 == target).sum().item()
            correct_resnet50 += (predicted_resnet50 == target).sum().item()

        #accuracy_alexnet = 100 * correct_alexnet / total
        accuracy_resnet15 = 100 * correct_resnet15 / total
        accuracy_resnet50 = 100 * correct_resnet50 / total

        #print(f"Epoch [{epoch+1}/{num_epochs}], Test Accuracy (AlexNet): {accuracy_alexnet:.2f}%")
        print(f"Epoch [{epoch+1}/{num_epochs}], Test Accuracy (ResNet-15): {accuracy_resnet15:.2f}%")
        print(f"Epoch [{epoch+1}/{num_epochs}], Test Accuracy (ResNet-50): {accuracy_resnet50:.2f}%")


RuntimeError: ignored